In [1]:
import pyomo.environ as pyo
import json
from datetime import timedelta
import polars as pl
from polars import selectors as cs
from polars  import col as c
import os 
import numpy as np
import math
import tqdm
from datetime import timedelta, datetime, timezone
# from optimization_model.optimizaztion_pipeline import first_stage_pipeline
from typing_extensions import Optional
from data_display.input_data_plots import plot_basin_height_volume_table
from baseline_model.optimization_results_processing import *
from data_display.baseline_plots import *
from utility.pyomo_preprocessing import *
from utility.input_data_preprocessing import *
from config import settings
from general_function import pl_to_dict, pl_to_dict_with_tuple, build_non_existing_dirs, generate_log, duckdb_to_dict

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objs as go
from plotly.graph_objects import Figure

from plotly.subplots import make_subplots

import networkx as nx
from data_display.baseline_plots import *
from baseline_model.baseline_input import BaseLineInput
from baseline_model.first_stage.first_stage_pipeline import BaselineFirstStage
from baseline_model.second_stage.second_stage_pipeline import BaselineSecondStage
COLORS = px.colors.qualitative.Plotly

log = generate_log(name="test")

os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = os.environ["HOME"] + "/gurobi_license/gurobi.lic"
volume_factor = 1e-6

In [2]:
output_file_names: dict[str, str] = json.load(open(settings.FILE_NAMES)) # type: ignore
smallflex_input_schema: SmallflexInputSchema = SmallflexInputSchema().duckdb_to_schema(file_path=output_file_names["duckdb_input"])

Read and validate tables from small_flex_input_data.db file: 100%|████████████████████████████████████████████████████| 15/15 [00:00<00:00, 25.46it/s]


In [3]:
PARALLEL = False
YEARS = [2020, 2021, 2022, 2023]
TURBINE_FACTORS = {0.75, 0.85, 0.95}

SIMULATION_SETTING = [
    {"quantile": 0, "buffer": 0.1, "powered_volume_enabled": True},
    # {"quantile": 0.15, "buffer": 0.3, "powered_volume_enabled": True},
    # {"quantile": 0.25, "buffer": 0.3, "powered_volume_enabled": False},
    # {"quantile": 0.15, "buffer": 0.3, "powered_volume_enabled": True, "global_price": True},
    # {"quantile": 0.25, "buffer": 0.3, "powered_volume_enabled": False, "global_price": True},
]

YEARS = 2021
TURBINE_FACTORS = 0.85
# hydro_power_mask = c("name").is_in(["Aegina discrete turbine"])
hydro_power_mask = c("name").is_in(["Aegina discrete turbine", "Aegina pump"])
hydro_power_mask = c("name").is_in(["Aegina discrete turbine", "Aegina continuous turbine", "Aegina pump"])


REAL_TIMESTEP = timedelta(hours=1)
FIRST_STAGE_TIMESTEP = timedelta(days=1)
SECOND_STAGE_TIME_SIM = timedelta(days=4)
TIME_LIMIT = 20 # in seconds
VOLUME_FACTOR = 1e-6

baseline_input: BaseLineInput = BaseLineInput(
    input_schema_file_name=output_file_names["duckdb_input"],
    real_timestep=REAL_TIMESTEP,
    year=YEARS,
    max_alpha_error=2,
    hydro_power_mask =hydro_power_mask,
    volume_factor=VOLUME_FACTOR
)
first_stage: BaselineFirstStage = BaselineFirstStage(
    nb_state= 3,
    input_instance=baseline_input,
    timestep=FIRST_STAGE_TIMESTEP,
    max_turbined_volume_factor=TURBINE_FACTORS
)
first_stage.solve_model()


# second_stage: BaselineSecondStage = BaselineSecondStage(
#         input_instance=baseline_input, 
#         first_stage=first_stage, 
#         timestep=SECOND_STAGE_TIME_SIM, 
#         time_limit=TIME_LIMIT,
#         model_nb=0,
#         nb_state=3,
#         is_parallel=PARALLEL,
#         **SIMULATION_SETTING[0]
#     )
# second_stage.solve_model()

Read and validate tables from small_flex_input_data.db file: 100%|████████████████████████████████████████████████████| 15/15 [00:00<00:00, 23.89it/s]
Solving first stage optimization problem: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.56it/s]


In [4]:
print(extract_optimization_results(first_stage.model_instance, "ancillary_power").to_pandas().to_string())

       T  CH  ancillary_power
0      0   1         0.000000
1      1   1         0.000000
2      2   1         0.000000
3      3   1         0.000000
4      4   1         0.000000
5      5   1         0.000000
6      6   1         3.500000
7      7   1         3.500000
8      8   1         0.000000
9      9   1         0.000000
10    10   1         3.500000
11    11   1         0.000000
12    12   1         0.000000
13    13   1         3.500000
14    14   1         0.000000
15    15   1         0.000000
16    16   1         0.000000
17    17   1         0.000000
18    18   1         0.000000
19    19   1         0.000000
20    20   1         0.000000
21    21   1         0.000000
22    22   1         0.000000
23    23   1         0.000000
24    24   1         0.000000
25    25   1         0.000000
26    26   1         0.000000
27    27   1         0.000000
28    28   1         0.000000
29    29   1         0.000000
30    30   1         0.000000
31    31   1         0.000000
32    32  

In [5]:
optimization_summary, combined_results = combine_second_stage_results(
    optimization_results= second_stage.optimization_results,
    powered_volume= second_stage.powered_volume,
    market_price=second_stage.market_price, 
    index=second_stage.index, 
    flow_to_vol_factor= second_stage.real_timestep.total_seconds() * second_stage.volume_factor)

NameError: name 'second_stage' is not defined

In [ ]:
plot_first_stage_result(
    simulation_results=first_stage.optimization_results, 
    time_divider=7
).show()

In [ ]:
first_stage.water_flow_factor

In [ ]:
first_stage.optimization_results

In [ ]:
extract_optimization_results(
            model_instance=first_stage.model_instance, var_name="spilled_volume"
        )


In [ ]:

turbined_power = extract_optimization_results(
        model_instance=model_instance, var_name="turbined_power"
    )

turbined_power = pivot_result_table(
    df = turbined_power, on="H", index=["T"],
    values="turbined_power")

simulation_results: pl.DataFrame = market_price\
    .join(basin_volume, on = "T", how="inner")\
    .join(turbined_volume, on = "T", how="inner")\
    .join(pumped_volume, on = "T", how="inner")\
    .join(pumped_power, on = "T", how="inner")\
    .join(turbined_power, on = "T", how="inner")\
    .with_columns(
        ((
            pl.sum_horizontal(cs.starts_with("turbined_power")) -
            pl.sum_horizontal(cs.starts_with("pumped_power"))
        ) * c("T").replace_strict(nb_hours_mapping, default=None) * c("market_price")).alias("income")
    )
    
hydro_name = list(map(str, list(model_instance.H))) # type: ignore

simulation_results = simulation_results.with_columns(
    pl.struct(cs.ends_with(hydro) & ~cs.starts_with("basin_volume"))
    .pipe(remove_suffix).alias("hydro_" + hydro) 
    for hydro in hydro_name
).select(    
    ~(cs.starts_with("turbined") | cs.starts_with("pumped")) # type: ignore
)
